In [1]:
import tweepy
import csv
import pandas as pd
import tqdm
import json
import time

In [2]:
bearer_token ="your_token"
auth = tweepy.OAuth2BearerHandler(bearer_token)
api = tweepy.API(auth, wait_on_rate_limit=True)

import requests
client = tweepy.Client(bearer_token=bearer_token,  
                       return_type = requests.Response,)
#                        wait_on_rate_limit=True)

In [14]:
# uk parlament list
list_members = api.get_list_members(list_id=217199644, count=1000)
list_id = []
for user in list_members:
    list_id.append(user._json['id'])

In [15]:
len(list_members)

535

In [17]:
for i, userID in enumerate(tqdm.tqdm(list_id)):
    keyword = "Brexit"
    query =f"{keyword} from:{userID} lang:en"
    tweets = client.search_all_tweets(query=query, 
                                      max_results=500, 
                                      start_time='2020-01-01T00:00:00.000Z', 
                                      end_time='2020-04-01T00:00:00.000Z', 
                                      tweet_fields = ['text']) 
    json_response = tweets_dict = tweets.json()
    json.dump(json_response, open("data/tweets0104/" + str(userID) + ".json", 'w'))
    time.sleep(2)
    if (i+1)%70 == 0:
        time.sleep(900)

100%|██████████| 535/535 [2:06:50<00:00, 14.23s/it]    


### Exctract beliefs

In [1]:
import json
import tqdm
import numpy as np 

from os import listdir
from os.path import isfile, join
from utils import roberta_sent_anal

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
data_path = 'data/tweets0104'
files = [f for f in listdir(data_path) if isfile(join(data_path, f))]
ids = [int(file[:-5]) for file in files]

In [8]:
sentiments_all = []
sentiments_avg = []
times_all = []
for user in tqdm.tqdm(ids):
    user_tweets = json.load(open(data_path + "/" + str(user) + ".json"))
    user_beliefs = []
    try:
        user_texts = [tweet['text'] for tweet in user_tweets['data']]
        user_times = [tweet['text'] for tweet in user_tweets['data']]
        sentiment_scores = roberta_sent_anal.eval_sentences_positive(user_texts)
        sentiments_all.append(np.array(sentiment_scores, dtype=np.float64))
        sentiments_avg.append(np.mean(sentiment_scores, dtype=np.float64))
        times_all.append(np.array(sentiment_scores, dtype=np.float64))
    except:
        sentiments_all.append([])
        sentiments_avg.append(0.5)
        times_all.append([])

100%|██████████| 535/535 [05:09<00:00,  1.73it/s]


In [12]:
dct_all = {}
for i, user in enumerate(tqdm.tqdm(ids)):
    dct_all[user] = list(sentiments_all[i])
    
dct_avg = {}
for i, user in enumerate(tqdm.tqdm(ids)):
    dct_avg[user] = float(sentiments_avg[i])
    
dct_times_all = {}
for i, user in enumerate(tqdm.tqdm(ids)):
    dct_times_all[user] = list(times_all[i])

100%|██████████| 535/535 [00:00<00:00, 622801.18it/s]


In [13]:
import json

# Serialize data into file:
json.dump(dct_all, open("data/belifs_all_dict0104.json", 'w'))
json.dump(dct_avg, open("data/belifs_avg_dict0104.json", 'w'))
json.dump(dct_times_all, open("data/times_all_dict0104.json", 'w'))